# Анализ базы данных сервиса для чтения книг

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Цель:**

Помочь бизнесу сформулировать ценностное предложение для нового продукта.

**Задача:**

Проанализировать базу данных.

***Описание данных***

**Таблица books**

Содержит данные о книгах:

`book_id` — идентификатор книги;

`author_id` — идентификатор автора;

`title` — название книги;

`num_pages` — количество страниц;

`publication_date` — дата публикации книги;

`publisher_id` — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:

`author_id` — идентификатор автора;

`author` — имя автора.

**Таблица publishers**

Содержит данные об издательствах:

`publisher_id` — идентификатор издательства;

`publisher` — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

`rating_id` — идентификатор оценки;

`book_id` — идентификатор книги;

`username` — имя пользователя, оставившего оценку;

`rating` — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах:

`review_id` — идентификатор обзора;

`book_id` — идентификатор книги;

`username` — имя автора обзора;

`text` — текст обзора.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Предварительный просмотр таблиц

Сначала исследуем таблицы - выведем их первые строки.

In [2]:
#таблица books
sql= '''
SELECT *
FROM books
LIMIT 1
'''

In [3]:
pd.io.sql.read_sql(sql, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [4]:
#дополнительно посмотрим, сколько всего книг имеется в базе.
sql= '''
SELECT count(b.book_id) AS count
FROM books b
'''

In [5]:
pd.io.sql.read_sql(sql, con = engine) 

,count
0,1000


In [6]:
#таблица authors
sql= '''
SELECT *
FROM authors
LIMIT 1
'''

In [7]:
pd.io.sql.read_sql(sql, con = engine) 

,author_id,author
0,1,A.S. Byatt


In [8]:
#таблица publishers
sql= '''
SELECT *
FROM publishers
LIMIT 1
'''

In [9]:
pd.io.sql.read_sql(sql, con = engine) 

,publisher_id,publisher
0,1,Ace


In [10]:
#таблица ratings
sql= '''
SELECT *
FROM ratings
LIMIT 1
'''

In [11]:
pd.io.sql.read_sql(sql, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [12]:
#таблица reviews
sql= '''
SELECT *
FROM reviews
LIMIT 1
'''

In [13]:
pd.io.sql.read_sql(sql, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


**Вывод:**

Посмотрели на первые строки таблиц, чтобы иметь представление, в каком виде информация в таблицах.

## Исследовательский анализ

Ответим на некоторые вопросы, которые могут помочь бизнесу в решении задачи.

### Сколько книг вышло после 1 января 2000 года?

In [14]:
sql = '''
SELECT COUNT(*) AS count_book
FROM books
WHERE publication_date > '2000-01-01'
'''

In [15]:
pd.io.sql.read_sql(sql, con = engine) 

,count_book
0,819


**Вывод:**

После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [16]:
sql= '''
WITH avg_rating AS
  (SELECT b.book_id,
          title,
          avg(r.rating) AS avg_rating
   FROM books b
   LEFT JOIN ratings r ON r.book_id = b.book_id
   GROUP BY b.book_id),
     reviews_count AS
  (SELECT b.book_id,
          title,
          count(rev.review_id) AS reviews_count
   FROM books b
   LEFT JOIN reviews rev ON rev.book_id = b.book_id
   GROUP BY b.book_id)
SELECT a.title, 
       c.reviews_count,
       a.avg_rating
FROM avg_rating a
JOIN reviews_count c ON a.book_id = c.book_id
ORDER BY a.avg_rating DESC
LIMIT 10
'''

In [17]:
pd.io.sql.read_sql(sql, con = engine) 

,title,reviews_count,avg_rating
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
1,Crucial Conversations: Tools for Talking When ...,2,5.0
2,Alas Babylon,2,5.0
3,Act of Treason (Mitch Rapp #9),2,5.0
4,Angels Fall,2,5.0
5,Captivating: Unveiling the Mystery of a Woman'...,2,5.0
6,A Woman of Substance (Emma Harte Saga #1),2,5.0
7,A Fistful of Charms (The Hollows #4),2,5.0
8,A Dirty Job (Grim Reaper #1),4,5.0
9,Dead Souls,2,5.0


In [18]:
sql= '''
WITH avg_rating AS
  (SELECT b.book_id,
          title,
          avg(r.rating) AS avg_rating
   FROM books b
   LEFT JOIN ratings r ON r.book_id = b.book_id
   GROUP BY b.book_id),
     reviews_count AS
  (SELECT b.book_id,
          title,
          count(rev.review_id) AS reviews_count
   FROM books b
   LEFT JOIN reviews rev ON rev.book_id = b.book_id
   GROUP BY b.book_id)
SELECT a.title, 
       c.reviews_count,
       a.avg_rating
FROM avg_rating a
JOIN reviews_count c ON a.book_id = c.book_id
ORDER BY a.avg_rating
LIMIT 10
'''

In [19]:
pd.io.sql.read_sql(sql, con = engine) 

,title,reviews_count,avg_rating
0,Harvesting the Heart,2,1.500000
1,Junky,2,2.000000
2,Drowning Ruth,3,2.000000
3,His Excellency: George Washington,2,2.000000
4,The World Is Flat: A Brief History of the Twen...,3,2.250000
5,The Mermaid Chair,3,2.333333
6,The Kitchen God's Wife,3,2.333333
7,Heretics of Dune (Dune Chronicles #5),2,2.500000
8,Amsterdam,2,2.500000
9,Dr. Faustus,2,2.500000


In [30]:
sql= '''
WITH avg_rating AS
  (SELECT b.book_id,
          title,
          avg(r.rating) AS avg_rating
   FROM books b
   LEFT JOIN ratings r ON r.book_id = b.book_id
   GROUP BY b.book_id),
     reviews_count AS
  (SELECT b.book_id,
          title,
          count(rev.review_id) AS reviews_count
   FROM books b
   LEFT JOIN reviews rev ON rev.book_id = b.book_id
   GROUP BY b.book_id)
SELECT a.title, 
       c.reviews_count,
       a.avg_rating
FROM avg_rating a
JOIN reviews_count c ON a.book_id = c.book_id
ORDER BY reviews_count DESC
LIMIT 10
'''

In [29]:
pd.io.sql.read_sql(sql, con = engine) 

,title,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


**Вывод:**

Отсортировав одну и ту же таблицу по убыванию и возрастанию, мы выяснили, что средний рейтинг может быть от 1,5 до 5,0. 
Максимальное количество обзоров - 7.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [22]:
sql = '''
SELECT
    p.publisher,
    COUNT(book_id) AS books_count
FROM books b
    LEFT JOIN publishers p ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher
ORDER BY books_count DESC
LIMIT 5
'''

In [23]:
pd.io.sql.read_sql(sql, con = engine) 

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


**Вывод:**

Издательство, которое выпустило наибольшее количество книг толще 50 страниц - Penguin Books

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [33]:
sql = '''
SELECT 
    a.author,
    AVG(r.rating) AS avg_rating
FROM 
    books b
LEFT JOIN ratings r on b.book_id = r.book_id
LEFT JOIN authors a on b.author_id = a.author_id
WHERE
    b.book_id IN (
        SELECT r.book_id
        FROM ratings r
        GROUP BY r.book_id
        HAVING COUNT(r.rating) > 50)
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 5
'''

In [34]:
pd.io.sql.read_sql(sql, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


**Вывод:**

Среди книг с более, чем 50 оценками, лидирует J.K. Rowling/Mary GrandPré. 

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [26]:
sql = '''
WITH
    ratings_q AS (
        SELECT
            r.username,
            COUNT(rating_id) AS ratings_count
        FROM ratings r
        GROUP BY r.username
        HAVING
            COUNT(rating_id) > 50
        ),
    reviews_q AS (
        SELECT
            rev.username,
            COUNT(review_id) AS reviews_count
        FROM reviews rev
        GROUP BY rev.username
    )
SELECT
    AVG(reviews_count) as avg_reviews_count
FROM ratings_q
    LEFT JOIN reviews_q ON reviews_q.username = ratings_q.username
'''

In [27]:
pd.io.sql.read_sql(sql, con = engine)

,avg_reviews_count
0,24.333333


**Вывод:**

В среднем количество обзоров от пользователей, которые поставили более 50 оценок - 24,33 обзора.

## Общий вывод

В данных представлена информация о 1000 книг, 819 из которых были выпущены после 1 января 2000 года.

Средний рейтинг может быть от 1,5 до 5,0. Максимальное количество обзоров - 7.

Издательство, которое выпустило наибольшее количество книг толще 50 страниц - Penguin Books.

Среди книг с более, чем 50 оценками, лидирует J.K. Rowling/Mary GrandPré.

В среднем количество обзоров от пользователей, которые поставили более 50 оценок - 24,33 обзора.